In [1]:
import numpy as np
import pandas as pd
import time
import gensim
import re

from gensim.models.word2vec import Word2Vec 
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Data Cleansing

In [2]:
df = pd.read_csv('../adam/datasets/scrape_5.12.csv')

pd.set_option('display.max_colwidth', -1)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
df.head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start
0,1234258409408602118,e.p.c.,"Did the first of several shopping runs, though I ended up walking out of CVS without buying anything. If the rate of people coughing at me is any indication, we’ll all have Covid-19 by Thursday. \n\nCover your fucking mouths when you cough.",2020-03-01 23:25:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
1,1234253374725459968,@geminiwoe,Looks like adios muchachos for the US #COVID #COVID19US #coronavirushttps://twitter.com/nytimes/status/1234171621696557057 …,2020-03-01 23:05:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
2,1234241890700218370,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,Humour..snl...on #COVID #coronavirususahttps://mashable.com/video/snl-coronavirus-democratic-candidates-cold-open/ …,2020-03-01 22:19:32,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
3,1234238588474331136,° ° °,had to go to three diff convenient stores to find real milk (not soy) and they were all out of the lower priced non organic stuff .. i buy organic so i was fine with it but wow. covid 19 panic buying begins in manhattan.,2020-03-01 22:06:25,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01
4,1234238537068883968,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,"#COVID #Coronovirius in nursing home setting , #Kirkland more than 50 with symptoms ..3/1/20https://www.statnews.com/2020/02/29/new-covid-19-death-raises-concerns-about-virus-spread-in-nursing-homes/ …",2020-03-01 22:06:12,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01


In [4]:
df.drop_duplicates(inplace=True)
df.shape

(114044, 10)

In [5]:
df['text'] = df['text'].astype(str).str.lower()
df['token_text'] = df['text'].str.replace('([^ a-zA-Z0-9])', '').str.replace('http\S+|www.\S+', '', case=False).replace('coronavirus', 'covid19')

stop = stopwords.words('english')
df['stop_text'] = df['token_text'].apply(lambda x: [item for item in str(x).split() 
                                                    if item not in stop])



#df['split_text'] = df['token_text'].astype(str).str.lower().str.split()

In [6]:
df['text'].head()

0    did the first of several shopping runs, though i ended up walking out of cvs without buying anything. if the rate of people coughing at me is any indication, we’ll all have covid-19 by thursday. \n\ncover your fucking mouths when you cough.
1    looks like adios muchachos for the us #covid #covid19us #coronavirushttps://twitter.com/nytimes/status/1234171621696557057 …                                                                                                                    
2    humour..snl...on #covid #coronavirususahttps://mashable.com/video/snl-coronavirus-democratic-candidates-cold-open/ …                                                                                                                            
3    had to go to three diff convenient stores to find real milk (not soy) and they were all out of the lower priced non organic stuff .. i buy organic so i was fine with it but wow. covid 19 panic buying begins in manhattan.                    
4    #covid #cor

In [7]:
df['stop_text'].head()

0    [first, several, shopping, runs, though, ended, walking, cvs, without, buying, anything, rate, people, coughing, indication, well, covid19, thursday, cover, fucking, mouths, cough]
1    [looks, like, adios, muchachos, us, covid, covid19us, coronavirus]                                                                                                                  
2    [humoursnlon, covid, coronavirususa]                                                                                                                                                
3    [go, three, diff, convenient, stores, find, real, milk, soy, lower, priced, non, organic, stuff, buy, organic, fine, wow, covid, 19, panic, buying, begins, manhattan]              
4    [covid, coronovirius, nursing, home, setting, kirkland, 50, symptoms, 3120]                                                                                                         
Name: stop_text, dtype: object

In [8]:
df['token_text'].head()

0    did the first of several shopping runs though i ended up walking out of cvs without buying anything if the rate of people coughing at me is any indication well all have covid19 by thursday cover your fucking mouths when you cough
1    looks like adios muchachos for the us covid covid19us coronavirus                                                                                                                                                                    
2    humoursnlon covid coronavirususa                                                                                                                                                                                                     
3    had to go to three diff convenient stores to find real milk not soy and they were all out of the lower priced non organic stuff  i buy organic so i was fine with it but wow covid 19 panic buying begins in manhattan               
4    covid coronovirius in nursing home setting  kirkland mo

## Word Vectorizer

In [9]:
sent = [row for row in df['stop_text']]

In [10]:
tweet_w2v = Word2Vec(size=1000, min_count=500, window=10)

In [11]:
tweet_w2v.build_vocab(sent)

In [12]:
tweet_w2v.most_similar('covid')

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('wants', 0.11421988904476166),
 ('things', 0.102251797914505),
 ('usa', 0.10049887001514435),
 ('false', 0.09476207196712494),
 ('check', 0.09043768793344498),
 ('wasnt', 0.08935615420341492),
 ('confirmed', 0.08576838672161102),
 ('long', 0.0759858638048172),
 ('fun', 0.07122430205345154),
 ('nursing', 0.07100999355316162)]

## Tweet Vectorizer

In [13]:
# Functions found in this cell were created by instructor Noah Christiansen
# He is great
# Define vectorization function
#word2vecmodel = tweet_w2v
def vectorize_corpus(keyword_list):    
    # Instantiate counter for number of words in keyword_list that exists
    n_words = 0
    # Create template for cumulative corpus vector sum
    corpus_vec_sum = np.zeros((1,1000))                 
    # Scan through each word in list
    for word in keyword_list:
        if word in tweet_w2v.wv.vocab:                    
            word_vec = tweet_w2v.wv.word_vec(word)
            #print(word_vec)
            n_words +=1                                
            corpus_vec_sum = corpus_vec_sum + word_vec 
    # Compute average vector by taking cumulative vector sum and dividing it by number of words traced
    corpus_avg_vec = corpus_vec_sum/n_words
    # Squeeze this N-dimensional nested array object into a 1-D array to streamline future processing
    corpus_avg_vec = np.squeeze(corpus_avg_vec)
    return(corpus_avg_vec)

#defining cosine similarity function
def cos_sim(vector_1, vector_2):
    dp = np.dot(vector_1, vector_2)
    magnitude_v1 = np.sqrt(np.dot(vector_1,vector_1))
    magnitude_v2 = np.sqrt(np.dot(vector_2,vector_2))
    return(dp/(magnitude_v1*magnitude_v2))


In [14]:
vect_tweets = [vectorize_corpus(tweet) for tweet in df['stop_text']]
vect_df = pd.DataFrame(vect_tweets)

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [16]:
print(df.shape)
print(vect_df.shape)

(114044, 12)
(114044, 1000)


In [34]:
join_df = df.join(vect_df)
join_df.dropna(inplace = True)
join_df.shape

(107432, 1012)

In [40]:
clean_vec_df= join_df.iloc[:, [i for i in range(13,1012)]]
clean_og_df = join_df.iloc[:, [i for i in range(0,12)]]
clean_vec_df.head()

,1,2,3,4,5,6,7,8,9,10,...,990,991,992,993,994,995,996,997,998,999
0,0.000133,0.000202,0.000003,0.000012,-0.000043,0.000128,0.000083,-0.000059,-0.000031,-0.000026,...,-0.000103,-0.000024,-0.000220,0.000021,0.000195,-1.707215e-07,0.000069,0.000071,-0.000023,-0.000003
1,0.000062,-0.000004,0.000180,0.000073,0.000011,0.000170,-0.000041,-0.000004,-0.000026,0.000029,...,-0.000114,0.000015,-0.000262,-0.000262,-0.000158,-9.967588e-05,-0.000175,-0.000069,-0.000198,0.000196
2,0.000038,-0.000358,0.000425,-0.000116,0.000196,0.000435,0.000033,-0.000208,-0.000100,0.000150,...,-0.000356,0.000171,-0.000246,-0.000413,0.000235,-1.871749e-04,0.000134,0.000468,0.000060,0.000048
3,0.000015,-0.000221,0.000112,0.000010,0.000066,-0.000087,-0.000101,-0.000084,0.000017,0.000076,...,-0.000110,-0.000036,-0.000071,0.000041,0.000081,5.050564e-05,0.000020,0.000044,0.000117,-0.000023
4,-0.000176,0.000164,0.000237,0.000004,0.000105,0.000194,-0.000059,-0.000239,0.000054,0.000006,...,-0.000158,0.000229,0.000082,-0.000208,0.000007,-1.101695e-04,0.000015,0.000194,0.000065,-0.000086


In [41]:
clean_og_df.head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start,token_text,stop_text
0,1234258409408602118,e.p.c.,"did the first of several shopping runs, though i ended up walking out of cvs without buying anything. if the rate of people coughing at me is any indication, we’ll all have covid-19 by thursday. \n\ncover your fucking mouths when you cough.",2020-03-01 23:25:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,did the first of several shopping runs though i ended up walking out of cvs without buying anything if the rate of people coughing at me is any indication well all have covid19 by thursday cover your fucking mouths when you cough,"[first, several, shopping, runs, though, ended, walking, cvs, without, buying, anything, rate, people, coughing, indication, well, covid19, thursday, cover, fucking, mouths, cough]"
1,1234253374725459968,@geminiwoe,looks like adios muchachos for the us #covid #covid19us #coronavirushttps://twitter.com/nytimes/status/1234171621696557057 …,2020-03-01 23:05:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,looks like adios muchachos for the us covid covid19us coronavirus,"[looks, like, adios, muchachos, us, covid, covid19us, coronavirus]"
2,1234241890700218370,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,humour..snl...on #covid #coronavirususahttps://mashable.com/video/snl-coronavirus-democratic-candidates-cold-open/ …,2020-03-01 22:19:32,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,humoursnlon covid coronavirususa,"[humoursnlon, covid, coronavirususa]"
3,1234238588474331136,° ° °,had to go to three diff convenient stores to find real milk (not soy) and they were all out of the lower priced non organic stuff .. i buy organic so i was fine with it but wow. covid 19 panic buying begins in manhattan.,2020-03-01 22:06:25,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,had to go to three diff convenient stores to find real milk not soy and they were all out of the lower priced non organic stuff i buy organic so i was fine with it but wow covid 19 panic buying begins in manhattan,"[go, three, diff, convenient, stores, find, real, milk, soy, lower, priced, non, organic, stuff, buy, organic, fine, wow, covid, 19, panic, buying, begins, manhattan]"
4,1234238537068883968,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,"#covid #coronovirius in nursing home setting , #kirkland more than 50 with symptoms ..3/1/20https://www.statnews.com/2020/02/29/new-covid-19-death-raises-concerns-about-virus-spread-in-nursing-homes/ …",2020-03-01 22:06:12,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,covid coronovirius in nursing home setting kirkland more than 50 with symptoms 3120,"[covid, coronovirius, nursing, home, setting, kirkland, 50, symptoms, 3120]"


In [42]:
dbscan = DBSCAN(metric='cosine', eps=0.01, min_samples=3, n_jobs = 1) # you can change these parameters, given just for example 
cluster_labels = dbscan.fit_predict(clean_vec_df) # where X - is your matrix, where each row corresponds to one document (line) from the docs, you need to cluster 

In [43]:
clean_og_df['cluster'] = cluster_labels

/opt/anaconda3/envs/dsi/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
clean_og_df['cluster'].value_counts()

-1       55443
 13      1314 
 4687    1022 
 987     288  
 3548    245  
        ...   
 8093    3    
 1886    3    
 5980    3    
 3871    3    
 3870    3    
Name: cluster, Length: 8796, dtype: int64

In [46]:
clean_og_df.head(50)

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start,token_text,stop_text,cluster
0,1234258409408602118,e.p.c.,"did the first of several shopping runs, though i ended up walking out of cvs without buying anything. if the rate of people coughing at me is any indication, we’ll all have covid-19 by thursday. \n\ncover your fucking mouths when you cough.",2020-03-01 23:25:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,did the first of several shopping runs though i ended up walking out of cvs without buying anything if the rate of people coughing at me is any indication well all have covid19 by thursday cover your fucking mouths when you cough,"[first, several, shopping, runs, though, ended, walking, cvs, without, buying, anything, rate, people, coughing, indication, well, covid19, thursday, cover, fucking, mouths, cough]",0
1,1234253374725459968,@geminiwoe,looks like adios muchachos for the us #covid #covid19us #coronavirushttps://twitter.com/nytimes/status/1234171621696557057 …,2020-03-01 23:05:10,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,looks like adios muchachos for the us covid covid19us coronavirus,"[looks, like, adios, muchachos, us, covid, covid19us, coronavirus]",1
2,1234241890700218370,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,humour..snl...on #covid #coronavirususahttps://mashable.com/video/snl-coronavirus-democratic-candidates-cold-open/ …,2020-03-01 22:19:32,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,humoursnlon covid coronavirususa,"[humoursnlon, covid, coronavirususa]",2
3,1234238588474331136,° ° °,had to go to three diff convenient stores to find real milk (not soy) and they were all out of the lower priced non organic stuff .. i buy organic so i was fine with it but wow. covid 19 panic buying begins in manhattan.,2020-03-01 22:06:25,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,had to go to three diff convenient stores to find real milk not soy and they were all out of the lower priced non organic stuff i buy organic so i was fine with it but wow covid 19 panic buying begins in manhattan,"[go, three, diff, convenient, stores, find, real, milk, soy, lower, priced, non, organic, stuff, buy, organic, fine, wow, covid, 19, panic, buying, begins, manhattan]",3
4,1234238537068883968,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,"#covid #coronovirius in nursing home setting , #kirkland more than 50 with symptoms ..3/1/20https://www.statnews.com/2020/02/29/new-covid-19-death-raises-concerns-about-virus-spread-in-nursing-homes/ …",2020-03-01 22:06:12,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,covid coronovirius in nursing home setting kirkland more than 50 with symptoms 3120,"[covid, coronovirius, nursing, home, setting, kirkland, 50, symptoms, 3120]",4
5,1234229558699753478,Yves Gebhardt,"from kofi anan, epidemics like #covid_19 are “problems without passports.” tall walls and immigration officials will not keep them out.. #funds4research\n#letscienceeducatelawmakers\n\ncc @drriveramindt @6dm4\n@cny_connections @emorycfarhttp://www.wbur.org/cognoscenti/2020/02/25/global-health-funding-coronavirus-pranay-sinha …",2020-03-01 21:30:32,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,from kofi anan epidemics like covid19 are problems without passports tall walls and immigration officials will not keep them out funds4researchletscienceeducatelawmakerscc drriveramindt 6dm4cnyconnections emorycfar,"[kofi, anan, epidemics, like, covid19, problems, without, passports, tall, walls, immigration, officials, keep, funds4researchletscienceeducatelawmakerscc, drriveramindt, 6dm4cnyconnections, emorycfar]",5
6,1234210797607378948,enigma4ever 🌹🆘 🌊 🕊️🍑👩‍⚕️💉😷,"as a nurse i am encouraging people to protect themselves, especially if they have immune issues & that includes wearing a mask, thorough hand & face washing ,avoiding crowds, spraying & wiping surfaces with alcohol/ chlorox products, & getting flu shot. #covid\n#coronavirus",2020-03-01 20:15:59,COVID,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,

In [51]:
mask = clean_og_df['cluster'] == 3548
clean_og_df[mask].head()

,tweet_id,username,text,tweet_date,search_term,city,lat,long,radius,query_start,token_text,stop_text,cluster
16718,1259992187468754945,Angelo Rosa,hell with what they say. prostitution is illegal and most of them spread deadly disease. you keep doing your job like you were trained.,2020-05-11 23:42:01,spread,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,hell with what they say prostitution is illegal and most of them spread deadly disease you keep doing your job like you were trained,"[hell, say, prostitution, illegal, spread, deadly, disease, keep, job, like, trained]",3548
16792,1257822374939906048,Thesikhmaster 🌈,we must seriously enforce the lockdown before it spread to other part of the world\n\nmany media may try to discourage us to enforce the lock down but we have large population and we cannot afford to let the virus to spread to the whole world,2020-05-05 23:59:58,spread,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,we must seriously enforce the lockdown before it spread to other part of the worldmany media may try to discourage us to enforce the lock down but we have large population and we cannot afford to let the virus to spread to the whole world,"[must, seriously, enforce, lockdown, spread, part, worldmany, media, may, try, discourage, us, enforce, lock, large, population, cannot, afford, let, virus, spread, whole, world]",3548
16806,1257817655144316934,Puneet Singh Ludu,herd immunity is not the only option. we did not reach here immunity for sars-1 or mers or swine flu.\nif we can control the spread herd immunity is meaningless and unnecessary,2020-05-05 23:41:12,spread,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,herd immunity is not the only option we did not reach here immunity for sars1 or mers or swine fluif we can control the spread herd immunity is meaningless and unnecessary,"[herd, immunity, option, reach, immunity, sars1, mers, swine, fluif, control, spread, herd, immunity, meaningless, unnecessary]",3548
16833,1257821049292496897,Marc Wolsky,#think2020 this experience has the pulse on the essential aspect of what is moving business and society forward in these challenging times. ibm is cutting edge with its duel profit and social focus!\nhttp://sciencefictionwave.com pic.twitter.com/ucrvdrwevk,2020-05-05 23:54:41,essential,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,think2020 this experience has the pulse on the essential aspect of what is moving business and society forward in these challenging times ibm is cutting edge with its duel profit and social focus,"[think2020, experience, pulse, essential, aspect, moving, business, society, forward, challenging, times, ibm, cutting, edge, duel, profit, social, focus]",3548
16895,1255952942940721154,Shayoni Mitra,please reconsider closure hours to daytime ones. people feel more vulnerable waiting for buses & shuttles on deserted streets at night. don’t make essential workers chose between safety and unaffordibilty/debt taking private transportation,2020-04-30 20:11:30,essential,Brooklyn,40.650002,-73.949997,10mi,2020-02-01,please reconsider closure hours to daytime ones people feel more vulnerable waiting for buses shuttles on deserted streets at night dont make essential workers chose between safety and unaffordibiltydebt taking private transportation,"[please, reconsider, closure, hours, daytime, ones, people, feel, vulnerable, waiting, buses, shuttles, deserted, streets, night, dont, make, essential, workers, chose, safety, unaffordibiltydebt, taking, private, transportation]",3548
